# Linear Regression Models

In [30]:
# Import the helper python files
import sys
import os

# Get the absolute path to the project's root directory (QRT)
# This goes 'up' one level from the 'models' folder
PROJECT_ROOT = os.path.abspath('..')

# Add the project root to the system path
if PROJECT_ROOT not in sys.path:
    sys.path.append(PROJECT_ROOT)

# Now you can import your helper files
import preprocess
import data_sourcing
import visualisations

print("Successfully imported helper modules from project root.")

Successfully imported helper modules from project root.


In [31]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

In [32]:
tech_sector_tickers = ['NVDA', 'AAPL', 'MSFT', 'GOOGL', 'META', 
                       'INTC', 'CSCO', 'ORCL', 'IBM', 'TSLA', 'AMZN', 
                       'QCOM', 'NFLX', 'ADBE', 'MS', 'ADP', 'LRCX']

data_sourcing.get_price_data(tech_sector_tickers, 
                                       start_date='2015-01-01', 
                                       end_date='2023-01-01')

[                       0%                       ]

Fetching price data for ['NVDA', 'AAPL', 'MSFT', 'GOOGL', 'META', 'INTC', 'CSCO', 'ORCL', 'IBM', 'TSLA', 'AMZN', 'QCOM', 'NFLX', 'ADBE', 'MS', 'ADP', 'LRCX']...


[*********************100%***********************]  17 of 17 completed
/Users/main/Desktop/QRT/data_sourcing.py:39: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  data = data.stack()


Successfully saved price data to /Users/main/Desktop/QRT/data/price_data.csv


close        high         low        open     volume
date       ticker                                                           
2015-01-02 AAPL     24.261049   24.729272   23.821673   24.718176  212818400
           ADBE     72.339996   73.199997   71.889999   72.699997    2349200
           ADP      65.986404   66.691505   65.376369   65.740809    1866600
           AMZN     15.426000   15.737500   15.348000   15.629000   55664000
           CSCO     19.815609   20.181635   19.650538   19.995033   22926500
...                       ...         ...         ...         ...        ...
2022-12-30 NFLX    294.880005  295.010010  283.220001  285.529999    7566900
           NVDA     14.600198   14.615183   14.219557   14.320462  310490000
           ORCL     78.760696   78.943774   77.816420   78.172929    5375700
           QCOM    103.229980  103.342652  100.675990  101.380215    5642300
           TSLA    123.180000  124.480003  119.750000  119.949997  157777300

[34238 rows x 5 columns]

In [28]:
tech_df = data_sourcing.load_price_data()

Loading data from /Users/main/Desktop/QRT/data/price_data.csv...


In [33]:
tech_df.head(17)

close       high        low       open     volume
date       ticker                                                       
2015-01-02 AAPL    24.261049  24.729272  23.821673  24.718176  212818400
           ADBE    72.339996  73.199997  71.889999  72.699997    2349200
           ADP     65.986404  66.691505  65.376369  65.740809    1866600
           AMD      2.670000   2.670000   2.670000   2.670000          0
           AMZN    15.426000  15.737500  15.348000  15.629000   55664000
           CSCO    19.815609  20.181635  19.650538  19.995033   22926500
           GOOGL   26.296133  26.606492  26.213203  26.447587   26480000
           IBM     98.664085  99.425106  98.018746  98.207481    5779673
           INTC    27.896458  28.510241  27.704650  28.134297   23605600
           LRCX     6.814786   6.878259   6.762463   6.850811    8306000
           META    77.969330  78.446392  77.223925  78.098538   18177500
           MS      28.715605  29.042005  28.374371  28.967822    7333800
           MSFT    39.933067  40.496708  39.745189  39.847669   27913900
           NFLX    49.848572  50.331429  48.731430  49.151428   13475000
           NVDA     0.483038   0.486638   0.475360   0.483038  113680000
           ORCL    37.583111  38.312219  37.277901  38.168094   15070200
           QCOM    54.612583  55.046369  54.222914  54.781688    6662700

In [35]:
tech_fundamental = data_sourcing.get_fundamental_data(tech_sector_tickers)
data_sourcing.load_fundamental_data()

Fetching fundamental data for 17 tickers...
Successfully saved fundamental data to /Users/main/Desktop/QRT/data/fundamental_data.csv
Loading data from /Users/main/Desktop/QRT/data/fundamental_data.csv...


,trailingPE,priceToBook,priceToSalesTtm,enterpriseToEbitda,returnOnEquity,returnOnAssets,operatingMargins,profitMargins,debtToEquity,currentRatio,quickRatio,earningsGrowth,revenueGrowth,beta,marketCap,sector,industry
ticker,,,,,,,,,,,,,,,,,
NVDA,53.105110,45.448578,NaN,46.033,1.09417,0.53094,0.60843,0.52414,10.584,4.214,3.488,0.612,0.556,2.269,4551184744448,Technology,Semiconductors
AAPL,35.908726,53.600475,NaN,27.937,1.71422,0.22964,0.31647,0.26915,152.411,0.893,0.771,0.912,0.079,1.109,3952969777152,Technology,Consumer Electronics
MSFT,35.275604,10.155098,NaN,22.309,0.32241,0.14656,0.48873,0.35707,33.154,1.401,1.254,0.127,0.184,1.065,3686273646592,Technology,Software - Infrastructure
GOOGL,27.528124,8.705394,NaN,23.295,0.35450,0.16276,0.30512,0.32233,11.424,1.747,1.563,0.353,0.159,1.082,3367128006656,Communication Services,Internet Content & Information
META,27.373950,8.025850,NaN,15.922,0.32643,0.18013,0.40075,0.30892,26.311,1.978,1.671,-0.826,0.262,1.272,1557257650176,Communication Services,Internet Content & Information
INTC,1257.833400,1.690636,NaN,20.473,0.00186,-0.00541,0.06284,0.00371,39.881,1.602,1.057,NaN,0.028,1.346,179845021696,Technology,Semiconductors
CSCO,27.816393,5.996432,NaN,19.106,0.22059,0.06342,0.21277,0.17969,63.282,0.998,0.802,0.186,0.076,0.869,279522705408,Technology,Communication Equipment
ORCL,54.831410,27.925194,NaN,32.455,0.69242,0.07152,0.31381,0.21080,452.534,0.618,0.498,-0.019,0.122,1.636,676835229696,Technology,Software - Infrastructure
IBM,36.581860,10.268850,NaN,19.352,0.30156,0.06350,0.17170,0.12090,237.831,0.932,0.775,NaN,0.091,0.688,286548688896,Technology,Information Technology Services
